
... ***CURRENTLY UNDER DEVELOPMENT*** ...


# Prepara output de las storms del emulador para hacer las RBFs de los casos propagados con SWAN

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.climate_emulator import Climate_Emulator
from teslakit.waves import AWL, Aggregate_WavesFamilies

from teslakit.plotting.outputs import Plot_FitSim_Histograms
from teslakit.plotting.extremes import Plot_FitSim_AnnualMax, Plot_FitSim_GevFit, Plot_Fit_QQ

from teslakit.plotting.waves import Plot_Waves_Histogram_FitSim


In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'

# climate change - S4
site = 'KWAJALEIN_CC_S4'

db_S4 = Database(p_data)
db_S4.SetSite(site)

## Load waves from the climate emulator

In [3]:
# Full simulation

CE = Climate_Emulator(db_S4.paths.site.EXTREMES.climate_emulator)

WVS_upd = CE.LoadSim_All(TCs=True)
WVS_upd = WVS_upd.to_xarray()

print(WVS_upd)

<xarray.Dataset>
Dimensions:      (index: 1032644)
Coordinates:
  * index        (index) int64 0 1 2 3 4 ... 1032640 1032641 1032642 1032643
Data variables:
    DWT          (index) float32 28.0 1.0 32.0 33.0 41.0 ... 13.0 28.0 20.0 20.0
    sea_Hs       (index) float32 1.5582333 2.434917 ... 1.810582 2.2550507
    sea_Tp       (index) float32 7.720595 8.870701 ... 8.210051 6.6658435
    sea_Dir      (index) float32 54.23647 70.633 ... 56.327915 67.612114
    swell_1_Hs   (index) float32 0.84767956 0.86827296 ... 0.3111764 0.42345622
    swell_1_Tp   (index) float32 8.847652 6.94969 ... 9.856896 11.085974
    swell_1_Dir  (index) float32 68.4031 97.40611 45.76165 ... 73.13391 72.43033
    swell_2_Hs   (index) float32 0.35236874 0.28442293 ... 0.45500863 0.26068825
    swell_2_Tp   (index) float32 11.0032215 12.334754 ... 11.017166 9.1800585
    swell_2_Dir  (index) float32 179.73355 171.34354 ... 179.41365 171.41524
    wind_speed   (index) float32 9.392085 6.3622866 ... 4.627558 7.380

In [4]:
#----------------------------------------------------------
# Poner NaN en WVS_upd_E donde Hs, Tp y Dir de Swell_1 y Swell_2 valen 0 (Hubo TC y sólo tenemos Sea family)

ind = np.where(WVS_upd.swell_1_Tp==0.0)[0]

vars = {'swell_1_Hs','swell_1_Tp','swell_1_Dir','swell_2_Hs','swell_2_Tp','swell_2_Dir'}
for var in vars:
    WVS_upd[var][ind] = np.nan


## Separate TC waves from RMC

In [5]:
WVS_sim_TCs = WVS_upd.copy(deep=True)
WVS_sim_RCM = WVS_upd.copy(deep=True)

WVS_sim_TCs = WVS_sim_TCs.where(np.isnan(WVS_sim_TCs.swell_1_Hs), drop=True)
print(WVS_sim_TCs)
print()

WVS_sim_RCM = WVS_sim_RCM.where(~np.isnan(WVS_sim_RCM.swell_1_Hs), drop=True)
print(WVS_sim_RCM)
print()

# Remove time variable
WVS_sim_RCM = WVS_sim_RCM.drop('time')
WVS_sim_TCs = WVS_sim_TCs.drop('time')

# save
from teslakit.io.aux_nc import StoreBugXdset 

StoreBugXdset(WVS_sim_RCM, '/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/EXTREMES/CE_all_sims_RCM.nc')  
StoreBugXdset(WVS_sim_TCs, '/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/EXTREMES/CE_all_sims_TCs.nc')  



<xarray.Dataset>
Dimensions:      (index: 8194)
Coordinates:
  * index        (index) int64 4 41 203 403 ... 1032036 1032201 1032284 1032325
Data variables:
    DWT          (index) float32 41.0 37.0 37.0 41.0 ... 38.0 37.0 37.0 37.0
    sea_Hs       (index) float32 4.2995853 1.8170794 ... 2.686796 1.7052553
    sea_Tp       (index) float32 10.114678 7.519681 ... 8.082397 7.3698897
    sea_Dir      (index) float32 103.02299 182.14929 ... 155.4255 148.87823
    swell_1_Hs   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_1_Tp   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_1_Dir  (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Hs   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Tp   (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    swell_2_Dir  (index) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    wind_speed   (index) float32 17.486217 9.5378 ...